# Preprocessing

## Imports

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import torch
import random
import cv2
from tqdm import tqdm
from matplotlib import pyplot as plt
import segmentation_models_pytorch as smp
import albumentations as album
from PIL import Image
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import random_split
from sklearn.metrics import accuracy_score
import plotly.express as px
import torchmetrics
from torchmetrics import MeanAbsolutePercentageError
from glob import glob

%matplotlib inline

## Global Variables

In [2]:
ROOT_DIR_PATH = os.path.abspath('..')

AADT_PATH = os.path.join(ROOT_DIR_PATH, 'data/ground_truth_data/')

AADT_PROCESSED_PATH = os.path.join(ROOT_DIR_PATH, 'data/ground_truth_data/aadt_processed.csv')

NN_MODEL_PATH = os.path.join(ROOT_DIR_PATH, "models/nn_aadt_model.pth")

## Data

In [3]:
pattern = os.path.join(ROOT_DIR_PATH, 'data/ground_truth_data/aadt_site_*.csv')

aadt_file_paths = [os.path.join(AADT_PATH, os.path.basename(x)) for x in glob(pattern)]
print("AADT files: {}".format(aadt_file_paths))

df = pd.read_csv(aadt_file_paths[0])

for i in range(1, len(aadt_file_paths)):
    df_aadt = pd.read_csv(aadt_file_paths[i])
    df = pd.concat([df, df_aadt], axis=0, ignore_index=True)

AADT files: ['/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data/aadt_site_7810_year_2019.csv', '/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data/aadt_site_7810_year_2018.csv', '/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data/aadt_site_4466_year_2018.csv', '/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data/aadt_site_4466_year_2019.csv', '/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data/aadt_site_2_year_2018.csv', '/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data/aadt_site_2_year_2019.csv']


In [4]:
df.head()

,Unnamed: 0,site_name,site_id,report_date,time_period_ending,time_interval,0-520cm,521-660cm,661-1160cm,1160+cm,avg_mph,total_volume,timestamp,year,month,day,hour,daily_count,aadt
0,0,6783/1,7810,2019-01-01,00:14:00,0,1,0,0,0,45.0,1,2019-01-01 00:14:00,2019,1,1,0,3998,6344.350685
1,1,6783/1,7810,2019-01-01,00:29:00,1,8,0,0,0,52.0,8,2019-01-01 00:29:00,2019,1,1,0,3998,6344.350685
2,2,6783/1,7810,2019-01-01,00:44:00,2,12,0,1,0,48.0,13,2019-01-01 00:44:00,2019,1,1,0,3998,6344.350685
3,3,6783/1,7810,2019-01-01,00:59:00,3,6,0,0,0,56.0,6,2019-01-01 00:59:00,2019,1,1,0,3998,6344.350685
4,4,6783/1,7810,2019-01-01,01:14:00,4,14,0,0,0,51.0,14,2019-01-01 01:14:00,2019,1,1,1,3998,6344.350685


## Corrupt values

In [5]:
df = df.dropna(axis=0)
print("df number of rows: {}".format(len(df)))
df.isnull().sum()

df number of rows: 4589


Unnamed: 0            0
site_name             0
site_id               0
report_date           0
time_period_ending    0
time_interval         0
0-520cm               0
521-660cm             0
661-1160cm            0
1160+cm               0
avg_mph               0
total_volume          0
timestamp             0
year                  0
month                 0
day                   0
hour                  0
daily_count           0
aadt                  0
dtype: int64

## Saving data

In [6]:
df.to_csv(AADT_PROCESSED_PATH)